# チュートリアル: ベクトル検索用に独自のドキュメントを準備する

In [ ]:
%pip install langchain pypdf langchain-openai --quiet

## 1. ドキュメントのアップロード
まず、`/docs` フォルダ内の既存のファイルを削除し、独自のPDFファイルを追加します。その後、以下のセルを実行してください。

In [ ]:
# docsディレクトリ内のすべてのファイルを処理するローダーを作成
import os
from langchain_community.document_loaders import PyPDFLoader

# docsディレクトリへのパス
docs_dir = "./docs"

# ディレクトリ内のすべてのファイルを取得
all_files = [os.path.join(docs_dir, f) for f in os.listdir(docs_dir) 
             if os.path.isfile(os.path.join(docs_dir, f))]

# ディレクトリ内の各ファイルを処理
documents = []
for file_path in all_files:
    try:
        loader = PyPDFLoader(
            file_path=file_path,
        )
        docs = loader.load()
        documents.extend(docs)
        print(f"{file_path}から{len(docs)}個のチャンクを読み込みました")
    except Exception as e:
        print(f"{file_path}の読み込みエラー: {e}")

print(f"合計{len(documents)}個のドキュメントチャンクを読み込みました")

## 2. ドキュメントのチャンク化
大きなドキュメントを小さなチャンクに分割して、埋め込み品質を向上させます。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ドキュメントを1000文字のチャンクに分割（200文字のオーバーラップ付き）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(documents)
print(f"{len(chunks)}個のチャンクを作成しました")

In [ ]:
# チャンクを表示
for chunk in chunks:
    print(chunk.page_content)
    print("-"*100)

## 3. 埋め込みの生成
OpenAI埋め込みを使用して、各チャンクをベクトルにエンコードします。

In [ ]:
from langchain_openai import OpenAIEmbeddings
# デフォルトのOpenAI埋め込みとして埋め込みを定義
embeddings = OpenAIEmbeddings()

## 4. Chromaに埋め込みを保存
Chromaベクトルストアを初期化し、ローカルに永続化します。

「OperationalError: attempt to write a readonly database」エラーが発生した場合は、カーネルを再起動してノートブックを再実行してください。

In [ ]:
from langchain.vectorstores import Chroma

# Chromaでベクトルストアを作成
vectordb = Chroma.from_documents(
    chunks,
    embedding=embeddings,
    persist_directory="db",
    collection_name="my_custom_index"
)
vectordb.persist()

## 5. 類似性検索の例
ベクトルストアで類似性検索クエリを実行します。

In [ ]:
# 類似性検索のテスト
query = "robotics"
results = vectordb.similarity_search(query, k=5)
for i, doc in enumerate(results):
    print(f"結果 {i+1}: {doc.page_content}...\n")